## filelecmp: сравнение файлов  

Модуль filecmp включает функции и класс, позволяющие сравнивать файлы и
каталоги, которые хранятся в файловой системе

In [22]:
import os

def mkfile(filename, body=None):
    with open(filename, 'w') as f:
        f.write(body or filename)
    return

def make_example_dir(top):
    if not os.path.exists(top):
        os.mkdir(top)
    curdir = os.getcwd()
    os.chdir(top)
    
    os.mkdir('dir1 ')
    os.mkdir('dir2 ')

    mkfile('dir1/file_only_in_dir1')
    mkfile('dir2/file_only_in_dir2')
             
    os.mkdir('dir1/dir_only_in_dir1')
    os.mkdir('dir2/dir_only_in_dir2')
             
    os.mkdir('dir1/common_dir')
    os.mkdir('dir2/common_dir')
             
    mkfile('dir1/common_file', 'this file is the same')
    mkfile('dir2/common_file', 'this file is the same')
             
    mkfile('dir1/not_the_same')
    mkfile('dir2/not_the_same')
             
    mkfile('dir1/file_in_dirl', 'This is a file in dir1')
    os.mkdir('dir2/file_in_dir1')
           
    os.chdir(curdir)
    return
           
if __name__ == '__main__' :
    os.chdir(os.path.dirname('Untitled1') or os.getcwd())
    make_example_dir('example')
    make_example_dir('example/dir1/common_dir')
    make_example_dir('example/dir2/common_dir')
    


В результате выполнения этого сценария в каталоге example создается дерево
файлов.

#### Сравнение файлов

Функция cmp () сравнивает два файла, принадлежащих файловой системе.

In [41]:
import filecmp

print('common_file :', end=' ')
print(filecmp.cmp('example/dir1/common_file',
                  'example/dir2/common_file'),
      end=' ')

print(filecmp.cmp('example/dir1/common_file',
                  'example/dir2/common_file',
                  shallow=False))
                  
print('not_the_same:', end=' ')
print(filecmp.cmp('example/dir1/not_the_same',
                  'example/dir2/not_the_same'),
      end=' ')

print(filecmp.cmp('example/dir1/not_the_same',
                  'example/dir1/not_the_same',
                  shallow=False))
      
print('identical :', end=' ')
print(filecmp.cmp('example/dir1/file_only_in_dir1',
                  'example/dir1/file_only_in_dir1'),
      end=' ')
      
print(filecmp.cmp('example/dir1/file_only_in_dir1',
                  'example/dir1/file_only_in_dir1',
                  shallow=False))

common_file : True True
not_the_same: False True
identical : True True


Для сравнения двух наборов файлов, находящихся в двух разных каталогах,
без применения рекурсии используйте функцию cmpfiles ()

In [44]:
import filecmp
import os

# Определение элементов, существующих в обоих каталогах
dl_contents = set(os.listdir('example/dir1'))
d2_contents = set(os.listdir('example/dir2'))
common = list(dl_contents & d2_contents)
common_files = [
    f
    for f in common
    if os.path.isfile(os.path.join('example/dir1', f))
]
print('Common files:', common_files)

# Сравнение каталогов
match, mismatch, errors = filecmp.cmpfiles(
    'example/dir1',
    'example/dir2',
    common_files,
)
print('Match :', match)
print('Mismatch :', mismatch)
print('Errors :', errors)

Common files: ['not_the_same', 'common_file']
Match : ['common_file']
Mismatch : ['not_the_same']
Errors : []


Функция cmpfiles () возвращает три списка имен файлов, которые соответствуют совпадающихМ и несовпадающим файлам, а также файлам, сравнение которых не может быть выполнено (ввиду отсутствия соответствующих полномочий
доступа или по иным причинам).

#### Сравнение каталогов 

Описанные до этого функции удобны для выполнения относительно простых
сравнений. Если требуется рекурсивное сравнение больших деревьев каталогов или проведение более полного катализа, гораздо лучше использовать класс dircmp. 

В простейших случаях для получения отчета о результатах сравнения
двух каталогов можно использовать функцию report ().

In [45]:
import filecmp

dc = filecmp.dircmp('example/dir1', 'example/dir2')
dc.report()

diff example/dir1 example/dir2
Only in example/dir1 : ['dir_only_in_dir1', 'file_in_dirl', 'file_only_in_dir1']
Only in example/dir2 : ['dir_only_in_dir2', 'file_in_dir1', 'file_only_in_dir2']
Identical files : ['common_file']
Differing files : ['not_the_same']
Common subdirectories : ['common_dir']


Для получения более подробной информации и выполнения рекурсивной опе-
рации сравнения следует использовать функцию report_full_closure ().

In [46]:
import filecmp

dc = filecmp.dircmp('example/dir1', 'example/dir2')
dc.report_full_closure()

diff example/dir1 example/dir2
Only in example/dir1 : ['dir_only_in_dir1', 'file_in_dirl', 'file_only_in_dir1']
Only in example/dir2 : ['dir_only_in_dir2', 'file_in_dir1', 'file_only_in_dir2']
Identical files : ['common_file']
Differing files : ['not_the_same']
Common subdirectories : ['common_dir']

diff example/dir1\common_dir example/dir2\common_dir
Common subdirectories : ['dir1', 'dir2']

diff example/dir1\common_dir\dir1 example/dir2\common_dir\dir1
Identical files : ['common_file', 'file_in_dirl', 'file_only_in_dir1', 'not_the_same']
Common subdirectories : ['common_dir', 'dir_only_in_dir1']

diff example/dir1\common_dir\dir1\common_dir example/dir2\common_dir\dir1\common_dir

diff example/dir1\common_dir\dir1\dir_only_in_dir1 example/dir2\common_dir\dir1\dir_only_in_dir1

diff example/dir1\common_dir\dir2 example/dir2\common_dir\dir2
Identical files : ['common_file', 'file_only_in_dir2', 'not_the_same']
Common subdirectories : ['common_dir', 'dir_only_in_dir2', 'file_in_dir1']


Вывод включает результаты сравнения всех параллельных подкаталогов.

#### Домашнее задание  

Создать каталог, содержащий 2 папки с произвольным набором файлов и сравнить их, используя функцию report()